In [1]:
import boto3
import pandas as pd
from io import StringIO , BytesIO
from datetime import datetime , timedelta

In [2]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('xetra-1234')

In [3]:
arg_date = '2022-12-23'

In [4]:
arg_date_dt = datetime.strptime(arg_date , '%Y-%m-%d').date()-timedelta(days=1)

In [5]:
arg_date_dt

datetime.date(2022, 12, 22)

In [6]:
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0],'%Y-%m-%d').date() >= arg_date_dt]

In [7]:
objects

[s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR00.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR01.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR02.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR03.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR04.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR05.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR06.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR07.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR08.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR09.csv'),
 s3.ObjectSummary(bucket_name='xetra-1234', key='2022-12-22/2022-12-22_BINS_XETR10.csv'),
 s3.Object

In [8]:
csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
data = StringIO(csv_obj_init)
df_init = pd.read_csv(data, delimiter=",")

In [9]:
df_init.columns

Index(['ISIN', 'Mnemonic', 'SecurityDesc', 'SecurityType', 'Currency',
       'SecurityID', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
       'EndPrice', 'TradedVolume', 'NumberOfTrades'],
      dtype='object')

In [15]:
df_all = pd.DataFrame(columns=df_init.columns)
dfs = []
for obj in objects:
    csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=",")
    dfs.append(df)

df_all = pd.concat(dfs , ignore_index=True)

In [16]:
df_all

,ISIN,Mnemonic,SecurityDesc,SecurityType,Currency,SecurityID,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A0HN5C6,DWNI,DEUTSCHE WOHNEN SE INH,Common stock,EUR,2504314,2022-12-22,08:00,36.090,36.090,35.830,35.860,912,7
1,DE000A1J5RX9,O2D,TELEFONICA DTLD HLDG NA,Common stock,EUR,2504492,2022-12-22,08:00,2.500,2.506,2.490,2.506,162400,27
2,DE000A1ML7J1,VNA,VONOVIA SE NA O.N.,Common stock,EUR,2504501,2022-12-22,08:00,45.410,45.410,45.210,45.240,47143,89
3,DE000A13SX22,HLE,HELLA GMBH+CO. KGAA O.N.,Common stock,EUR,2504580,2022-12-22,08:00,62.000,62.080,62.000,62.080,4591,2
4,DE000PAT1AG3,PAT,PATRIZIA AG NA ON,Common stock,EUR,2504817,2022-12-22,08:00,17.700,17.700,17.520,17.520,953,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1352560,GB00BLD4ZP54,CLTC,COINSHARES DIG.SEC.OEND,ETN,EUR,6479084,2022-12-31,16:46,19.324,19.324,19.324,19.324,0,2
1352561,LU1923627332,RUSL,MUL-LYX.MSCI RUSSI.DIS.LS,ETF,EUR,5424594,2022-12-31,16:52,12.400,12.400,12.400,12.400,2645,2
1352562,US98956P1021,ZIM,ZIMMER BIOMET HLDGS DL-01,Common stock,EUR,4582018,2022-12-31,20:30,113.100,113.100,113.100,113.100,0,1
1352563,US9224171002,VEO,"VEECO INSTRUMENTS DL-,01",Common stock,EUR,6198311,2022-12-31,20:30,24.600,24.600,24.600,24.600,0,1


In [17]:
columns = ['ISIN',  'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice','EndPrice', 'TradedVolume', 'NumberOfTrades']

In [18]:
df_all = df_all.loc[:, columns]

In [19]:
df_all.head()

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades
0,DE000A0HN5C6,2022-12-22,08:00,36.09,36.090,35.83,35.860,912,7
1,DE000A1J5RX9,2022-12-22,08:00,2.50,2.506,2.49,2.506,162400,27
2,DE000A1ML7J1,2022-12-22,08:00,45.41,45.410,45.21,45.240,47143,89
3,DE000A13SX22,2022-12-22,08:00,62.00,62.080,62.00,62.080,4591,2
4,DE000PAT1AG3,2022-12-22,08:00,17.70,17.700,17.52,17.520,953,9


In [20]:
df_all.dropna(inplace=True)

In [21]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352565 entries, 0 to 1352564
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   ISIN            1352565 non-null  object 
 1   Date            1352565 non-null  object 
 2   Time            1352565 non-null  object 
 3   StartPrice      1352565 non-null  float64
 4   MaxPrice        1352565 non-null  float64
 5   MinPrice        1352565 non-null  float64
 6   EndPrice        1352565 non-null  float64
 7   TradedVolume    1352565 non-null  object 
 8   NumberOfTrades  1352565 non-null  object 
dtypes: float64(4), object(5)
memory usage: 92.9+ MB


In [22]:
df_all.describe()

,StartPrice,MaxPrice,MinPrice,EndPrice
count,1.352565e+06,1.352565e+06,1.352565e+06,1.352565e+06
mean,8.957918e+01,8.960980e+01,8.954861e+01,8.957931e+01
std,5.749694e+02,5.749864e+02,5.749513e+02,5.749684e+02
min,6.000000e-04,6.000000e-04,6.000000e-04,6.000000e-04
25%,1.616200e+01,1.618000e+01,1.615000e+01,1.616200e+01
50%,4.069000e+01,4.070000e+01,4.067000e+01,4.069000e+01
75%,8.830000e+01,8.834000e+01,8.826800e+01,8.830000e+01
max,1.148280e+05,1.148280e+05,1.148280e+05,1.148280e+05


In [23]:
df_all.shape

(1352565, 9)

## Get the opening price per ISIN and day

In [24]:
df_all['opening_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("first")

In [25]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,opening_price
39,AT0000A0E9W5,2022-12-22,08:02,13.90,13.90,13.76,13.88,17490,17,13.90
202,AT0000A0E9W5,2022-12-22,08:03,13.88,13.88,13.88,13.88,349,1,13.90
300,AT0000A0E9W5,2022-12-22,08:04,13.98,13.98,13.89,13.89,1443,2,13.90
2318,AT0000A0E9W5,2022-12-22,08:05,13.88,14.00,13.88,14.00,5995,3,13.90
2665,AT0000A0E9W5,2022-12-22,08:06,14.01,14.01,14.01,14.01,300,1,13.90
...,...,...,...,...,...,...,...,...,...,...
1347733,AT0000A0E9W5,2022-12-31,16:26,14.04,14.04,14.04,14.04,1129,10,13.88
1348109,AT0000A0E9W5,2022-12-31,16:27,14.03,14.14,14.02,14.04,6627,19,13.88
1348403,AT0000A0E9W5,2022-12-31,16:28,14.04,14.04,14.03,14.03,2003,10,13.88
1348754,AT0000A0E9W5,2022-12-31,16:29,14.08,14.08,14.00,14.00,354,5,13.88


## Get closing price per ISIN and Day

In [26]:
df_all['closing_price'] = df_all.sort_values('Time').groupby(['ISIN','Date'])['StartPrice'].transform("last")

In [27]:
df_all[df_all['ISIN']=='AT0000A0E9W5']

,ISIN,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,opening_price,closing_price
39,AT0000A0E9W5,2022-12-22,08:02,13.90,13.90,13.76,13.88,17490,17,13.90,14.48
202,AT0000A0E9W5,2022-12-22,08:03,13.88,13.88,13.88,13.88,349,1,13.90,14.48
300,AT0000A0E9W5,2022-12-22,08:04,13.98,13.98,13.89,13.89,1443,2,13.90,14.48
2318,AT0000A0E9W5,2022-12-22,08:05,13.88,14.00,13.88,14.00,5995,3,13.90,14.48
2665,AT0000A0E9W5,2022-12-22,08:06,14.01,14.01,14.01,14.01,300,1,13.90,14.48
...,...,...,...,...,...,...,...,...,...,...,...
1347733,AT0000A0E9W5,2022-12-31,16:26,14.04,14.04,14.04,14.04,1129,10,13.88,14.08
1348109,AT0000A0E9W5,2022-12-31,16:27,14.03,14.14,14.02,14.04,6627,19,13.88,14.08
1348403,AT0000A0E9W5,2022-12-31,16:28,14.04,14.04,14.03,14.03,2003,10,13.88,14.08
1348754,AT0000A0E9W5,2022-12-31,16:29,14.08,14.08,14.00,14.00,354,5,13.88,14.08


## Aggregrations

In [28]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_euro=('opening_price','min'),closing_price_euro=('closing_price','min'), minimun_price_euro=('MinPrice','min'),maximun_price_euro=('MaxPrice','max'), daily_traded_volume=('TradedVolume','sum'))

In [29]:
df_all

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume
0,AT000000STR1,2022-12-22,36.4000,36.8500,36.4000,36.8500,1475
1,AT000000STR1,2022-12-23,37.0000,37.9500,37.0000,37.9500,396
2,AT000000STR1,2022-12-24,36.0500,36.1000,35.6500,37.0500,1838
3,AT000000STR1,2022-12-25,36.1000,37.7000,36.1000,37.7000,2864
4,AT000000STR1,2022-12-26,36.1000,37.7000,36.1000,37.7000,2864
...,...,...,...,...,...,...,...
32099,XS2434891219,2022-12-27,3.4404,3.5034,3.4404,3.5034,0
32100,XS2434891219,2022-12-28,3.4412,3.6620,3.4184,3.6620,0
32101,XS2434891219,2022-12-29,3.4412,3.6620,3.4184,3.6620,0
32102,XS2434891219,2022-12-30,3.4412,3.6620,3.4184,3.6620,0


## Percentage Change previous closing price

In [30]:
df_all['previous_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_euro'].shift(1)

In [31]:
df_all

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,previous_closing_price
0,AT000000STR1,2022-12-22,36.4000,36.8500,36.4000,36.8500,1475,NaN
1,AT000000STR1,2022-12-23,37.0000,37.9500,37.0000,37.9500,396,36.8500
2,AT000000STR1,2022-12-24,36.0500,36.1000,35.6500,37.0500,1838,37.9500
3,AT000000STR1,2022-12-25,36.1000,37.7000,36.1000,37.7000,2864,36.1000
4,AT000000STR1,2022-12-26,36.1000,37.7000,36.1000,37.7000,2864,37.7000
...,...,...,...,...,...,...,...,...
32099,XS2434891219,2022-12-27,3.4404,3.5034,3.4404,3.5034,0,3.5034
32100,XS2434891219,2022-12-28,3.4412,3.6620,3.4184,3.6620,0,3.5034
32101,XS2434891219,2022-12-29,3.4412,3.6620,3.4184,3.6620,0,3.6620
32102,XS2434891219,2022-12-30,3.4412,3.6620,3.4184,3.6620,0,3.6620


In [32]:
df_all['percentage_change_prev%'] = ((df_all['closing_price_euro']-df_all['previous_closing_price'])/df_all['previous_closing_price'])*100

In [33]:
df_all

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,previous_closing_price,percentage_change_prev%
0,AT000000STR1,2022-12-22,36.4000,36.8500,36.4000,36.8500,1475,NaN,NaN
1,AT000000STR1,2022-12-23,37.0000,37.9500,37.0000,37.9500,396,36.8500,2.985075
2,AT000000STR1,2022-12-24,36.0500,36.1000,35.6500,37.0500,1838,37.9500,-4.874835
3,AT000000STR1,2022-12-25,36.1000,37.7000,36.1000,37.7000,2864,36.1000,4.432133
4,AT000000STR1,2022-12-26,36.1000,37.7000,36.1000,37.7000,2864,37.7000,0.000000
...,...,...,...,...,...,...,...,...,...
32099,XS2434891219,2022-12-27,3.4404,3.5034,3.4404,3.5034,0,3.5034,0.000000
32100,XS2434891219,2022-12-28,3.4412,3.6620,3.4184,3.6620,0,3.5034,4.527031
32101,XS2434891219,2022-12-29,3.4412,3.6620,3.4184,3.6620,0,3.6620,0.000000
32102,XS2434891219,2022-12-30,3.4412,3.6620,3.4184,3.6620,0,3.6620,0.000000


In [34]:
df_all.drop(columns=['previous_closing_price'],inplace=True)

In [35]:
df_all = df_all.round(decimals=2)

In [36]:
df_all = df_all[df_all.Date >= arg_date]

In [37]:
df_all

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,percentage_change_prev%
1,AT000000STR1,2022-12-23,37.00,37.95,37.00,37.95,396,2.99
2,AT000000STR1,2022-12-24,36.05,36.10,35.65,37.05,1838,-4.87
3,AT000000STR1,2022-12-25,36.10,37.70,36.10,37.70,2864,4.43
4,AT000000STR1,2022-12-26,36.10,37.70,36.10,37.70,2864,0.00
5,AT000000STR1,2022-12-27,36.10,37.70,36.10,37.70,2864,0.00
...,...,...,...,...,...,...,...,...
32099,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
32100,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
32101,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
32102,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00


In [38]:
## Write to s3

In [44]:
key = 'xetra_daiy_report_' + datetime.today().strftime('%Y-%m-%m_%H-%M-%S') + '.parquet'

In [45]:
outbuffer =BytesIO()
df_all.to_parquet(outbuffer , index=False)
target_bucket = s3.Bucket('targetbucket-1234')
target_bucket.put_object(Body = outbuffer.getvalue() , Key = key)

s3.Object(bucket_name='targetbucket-1234', key='xetra_daiy_report_2023-04-04_02-13-42.parquet')

In [46]:
## checking

In [47]:
for obj in target_bucket.objects.all():
    print(obj.key)

xetra_daiy_report2023-04-04_02-07-12.parquet
xetra_daiy_report_2023-04-04_02-13-42.parquet


In [50]:
prq_obj = target_bucket.Object(key='xetra_daiy_report_2023-04-04_02-13-42.parquet').get().get('Body').read()

In [51]:
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [52]:
df_report

,ISIN,Date,opening_price_euro,closing_price_euro,minimun_price_euro,maximun_price_euro,daily_traded_volume,percentage_change_prev%
0,AT000000STR1,2022-12-23,37.00,37.95,37.00,37.95,396,2.99
1,AT000000STR1,2022-12-24,36.05,36.10,35.65,37.05,1838,-4.87
2,AT000000STR1,2022-12-25,36.10,37.70,36.10,37.70,2864,4.43
3,AT000000STR1,2022-12-26,36.10,37.70,36.10,37.70,2864,0.00
4,AT000000STR1,2022-12-27,36.10,37.70,36.10,37.70,2864,0.00
...,...,...,...,...,...,...,...,...
28846,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
28847,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
28848,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
28849,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
